# **Задача детекции аномалий во временном ряде**

## Необходимые импорты:

In [2]:
import warnings
warnings.filterwarnings('ignore')

from orion import Orion

import numpy as np
import pandas as pd
from numpy import percentile

import plotly.graph_objects as go
import plotly.figure_factory as ff

import keras
from keras import layers

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors, LocalOutlierFactor

from adtk.data import validate_series
from adtk.detector import MinClusterDetector, OutlierDetector
from adtk.detector import PcaAD

2024-05-10 06:18:55.648726: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 06:18:55.648811: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 06:18:55.648849: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## 1. Unsupervised подходы

In [3]:
ts_no_label = pd.read_csv('/kaggle/input/nab/realAdExchange/realAdExchange/exchange-2_cpc_results.csv', parse_dates=['timestamp'])
ts_no_label.head()

,timestamp,value
0,2011-07-01 00:00:01,0.081965
1,2011-07-01 01:00:01,0.098972
2,2011-07-01 02:00:01,0.065314
3,2011-07-01 03:00:01,0.070663
4,2011-07-01 04:00:01,0.102490


In [4]:
def overview(df: pd.DataFrame, timestamp_col: str=None) -> None:
    print('Null Count:\n', df.isnull().sum(), '\n')
    print('Data Types:\n', df.dtypes)
    
    if timestamp_col is not None:
        print('\nDate Range:\n\nStart:\t', df[timestamp_col].min())
        print('End:\t', df[timestamp_col].max())
        print('Days:\t',(df[timestamp_col].max() - df[timestamp_col].min()))

In [5]:
overview(ts_no_label, timestamp_col='timestamp')

Null Count:
 timestamp    0
value        0
dtype: int64 

Data Types:
 timestamp    datetime64[ns]
value               float64
dtype: object

Date Range:

Start:	 2011-07-01 00:00:01
End:	 2011-09-07 15:00:01
Days:	 68 days 15:00:00


In [6]:
hourly_data = ts_no_label.set_index('timestamp').resample('H').mean()
daily_data = ts_no_label.set_index('timestamp').resample('D').mean()
weekly_data = ts_no_label.set_index('timestamp').resample('W').mean()

fig_hourly = go.Figure()
fig_hourly.add_trace(go.Scatter(x=hourly_data.index, y=hourly_data['value'],
                    mode='lines',
                    name='Hourly',
                    hovertemplate='%{y:.2f}<extra></extra>'))

fig_daily = go.Figure()
fig_daily.add_trace(go.Scatter(x=daily_data.index, y=daily_data['value'],
                    mode='lines',
                    name='Daily',
                    hovertemplate='%{y:.2f}<extra></extra>'))

fig_weekly = go.Figure()
fig_weekly.add_trace(go.Scatter(x=weekly_data.index, y=weekly_data['value'],
                    mode='lines',
                    name='Weekly',
                    hovertemplate='%{y:.2f}<extra></extra>'))

fig_hourly.update_layout(title="Exchange Hourly", 
                         xaxis_title="", 
                         yaxis_title="Exchange results", 
                         showlegend=True, 
                         height=500, 
                         hovermode='x', 
                         hoverlabel=dict(bgcolor="white", font_size=12, font_family="Rockwell"))

fig_daily.update_layout(title="Exchange Daily", 
                        xaxis_title="", 
                        yaxis_title="Exchange results", 
                        showlegend=True, 
                        height=500, 
                        hovermode='x', 
                        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Rockwell"))

fig_weekly.update_layout(title="Exchange Weekly", 
                         xaxis_title="Date", 
                         yaxis_title="Exchange results", 
                         showlegend=True, 
                         height=500, 
                         hovermode='x', 
                         hoverlabel=dict(bgcolor="white", font_size=12, font_family="Rockwell"))

fig_hourly.show()
fig_daily.show()
fig_weekly.show()

In [7]:
ts_no_label_hourly = ts_no_label.set_index('timestamp').resample('H').mean().reset_index()
ts_no_label_daily = ts_no_label.set_index('timestamp').resample('D').mean().reset_index()
ts_no_label_weekly = ts_no_label.set_index('timestamp').resample('W').mean().reset_index()

In [8]:
for ts_ in [ts_no_label_hourly, ts_no_label_daily, ts_no_label_weekly]:
    
    ts_['Weekday'] = (pd.Categorical(ts_['timestamp'].dt.strftime('%A'),
                                     categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday', 'Saturday', 'Sunday'])
                     )
    
    ts_['Hour'] = ts_['timestamp'].dt.hour
    ts_['Day'] = ts_['timestamp'].dt.weekday
    ts_['Month'] = ts_['timestamp'].dt.month
    ts_['Year'] = ts_['timestamp'].dt.year
    ts_['Month_day'] = ts_['timestamp'].dt.day
    ts_['Lag'] = ts_['value'].shift(1)
    ts_['Rolling_Mean'] = ts_['value'].rolling(7, min_periods=1).mean()
    ts_ = ts_.dropna()

In [9]:
fig = ff.create_distplot([ts_no_label['value']], 
                         ['Overall Value Distribution'], 
                         bin_size=0.1, 
                         show_rug=False, 
                         histnorm='probability density')

fig.update_layout(title="Overall Value Distribution", 
                  xaxis_title="Value", 
                  yaxis_title="Density", 
                  height=500, 
                  showlegend=True, 
                  hovermode='closest')

for trace in fig['data']:
    trace['marker']['opacity'] = 0.5

fig.show()

In [10]:
by_weekday = ts_no_label_hourly.groupby(['Hour', 'Weekday']).mean()['value'].unstack()

fig = go.Figure()
for weekday, values in by_weekday.items():
    fig.add_trace(go.Scatter(x=values.index, y=values, mode='lines', name=weekday))

fig.update_layout(title="Exchange Density by Day & Hour", 
                  xaxis_title="Hour", 
                  yaxis_title="Exchange", 
                  height=500, 
                  hovermode='x', 
                  showlegend=True)

fig.show()

In [11]:
average_demand_by_day = ts_no_label_hourly.groupby('Weekday').mean()['value']

fig = go.Figure(go.Bar(
    x=average_demand_by_day.index,
    y=average_demand_by_day.values,
    hovertemplate='Exchange: %{y}<extra></extra>',
))

fig.update_layout(
    title="Exchange Results by Day",
    xaxis_title="",
    yaxis_title="Exchange",
    height=500,
    showlegend=False,
    hovermode='x',
    bargap=0.3,
)

fig.show()

In [12]:
by_weekday = ts_no_label_hourly.groupby(['Hour', 'Weekday']).mean()['value'].unstack()

fig = go.Figure()
for weekday in by_weekday.columns:
    fig.add_trace(go.Scatter(
        x=by_weekday.index,
        y=by_weekday[weekday],
        mode='lines',
        name=weekday,
        hovertemplate='Hour: %{x}<br>Exchange: %{y}<extra></extra>'
    ))

fig.update_layout(
    title="Average Exchange by Day & Hour",
    xaxis_title="Hour",
    yaxis_title="Exchange",
    height=500,
    showlegend=True,
    hovermode='closest',
    legend=dict(
        title="Weekday",
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [13]:
ts_no_label_hourly = (ts_no_label_hourly
             .join(ts_no_label_hourly.groupby(['Hour','Weekday'])['value'].mean(),
                   on = ['Hour', 'Weekday'], rsuffix='_Average')
            )

ts_no_label_daily = (ts_no_label_daily
            .join(ts_no_label_daily.groupby(['Hour','Weekday'])['value'].mean(),
                  on = ['Hour', 'Weekday'], rsuffix='_Average')
           )

ts_no_label_hourly.tail()

,timestamp,value,Weekday,Hour,Day,Month,Year,Month_day,Lag,Rolling_Mean,value_Average
1643,2011-09-07 11:00:00,0.094662,Wednesday,11,2,9,2011,7,0.135588,0.115477,0.121750
1644,2011-09-07 12:00:00,0.097657,Wednesday,12,2,9,2011,7,0.094662,0.116625,0.118341
1645,2011-09-07 13:00:00,0.096201,Wednesday,13,2,9,2011,7,0.097657,0.114500,0.119499
1646,2011-09-07 14:00:00,0.085386,Wednesday,14,2,9,2011,7,0.096201,0.112265,0.118467
1647,2011-09-07 15:00:00,0.109327,Wednesday,15,2,9,2011,7,0.085386,0.106224,0.123286


### 1.1 Isolation Forest

In [14]:
ts_no_label_hourly.dropna(inplace=True)

ts_no_label_daily_model_data = ts_no_label_daily[['value', 
                                                  'Hour', 
                                                  'Day', 
                                                  'Month', 
                                                  'Month_day', 
                                                  'Rolling_Mean']].dropna()

model_data = ts_no_label_hourly[['value', 
                                 'Hour', 
                                 'Day', 
                                 'Month_day', 
                                 'Month', 
                                 'Rolling_Mean', 
                                 'Lag', 
                                 'timestamp']].set_index('timestamp').dropna()

model_data.head()

,value,Hour,Day,Month_day,Month,Rolling_Mean,Lag
timestamp,,,,,,,
2011-07-01 01:00:00,0.098972,1,4,1,7,0.090468,0.081965
2011-07-01 02:00:00,0.065314,2,4,1,7,0.082084,0.098972
2011-07-01 03:00:00,0.070663,3,4,1,7,0.079228,0.065314
2011-07-01 04:00:00,0.102490,4,4,1,7,0.083881,0.070663
2011-07-01 05:00:00,0.123395,5,4,1,7,0.090466,0.102490


In [15]:
SEED = sum(ord(ch) for ch in 'MISISFOUNDHACK')
SEED

1048

In [16]:
def run_isolation_forest(model_data: pd.DataFrame, 
                         contamination=0.005, 
                         n_estimators=200, 
                         max_samples=0.7) -> pd.DataFrame:
    
    IF = (IsolationForest(random_state=SEED,
                          contamination=contamination,
                          n_estimators=n_estimators,
                          max_samples=max_samples)
         )
    
    IF.fit(model_data)
    
    output = pd.Series(IF.predict(model_data)).apply(lambda x: 1 if x == -1 else 0)
    
    score = IF.decision_function(model_data)
    
    return output, score

In [17]:
outliers, score = run_isolation_forest(model_data)

In [18]:
ts_no_label_hourly = (ts_no_label_hourly
                      .assign(Outliers = outliers)
                      .assign(Score = score)
                     )

ts_no_label_hourly

,timestamp,value,Weekday,Hour,Day,Month,Year,Month_day,Lag,Rolling_Mean,value_Average,Outliers,Score
1,2011-07-01 01:00:00,0.098972,Friday,1,4,7,2011,1,0.081965,0.090468,0.069339,0.0,0.105570
2,2011-07-01 02:00:00,0.065314,Friday,2,4,7,2011,1,0.098972,0.082084,0.061827,0.0,0.095479
3,2011-07-01 03:00:00,0.070663,Friday,3,4,7,2011,1,0.065314,0.079228,0.056796,0.0,0.125900
4,2011-07-01 04:00:00,0.102490,Friday,4,4,7,2011,1,0.070663,0.083881,0.075489,0.0,0.110238
5,2011-07-01 05:00:00,0.123395,Friday,5,4,7,2011,1,0.102490,0.090466,0.092917,0.0,0.118445
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643,2011-09-07 11:00:00,0.094662,Wednesday,11,2,9,2011,7,0.135588,0.115477,0.121750,NaN,0.084850
1644,2011-09-07 12:00:00,0.097657,Wednesday,12,2,9,2011,7,0.094662,0.116625,0.118341,NaN,0.102500
1645,2011-09-07 13:00:00,0.096201,Wednesday,13,2,9,2011,7,0.097657,0.114500,0.119499,NaN,0.114009
1646,2011-09-07 14:00:00,0.085386,Wednesday,14,2,9,2011,7,0.096201,0.112265,0.118467,NaN,0.095052


In [19]:
def outliers(thresh):
    print(f'Number of Outliers below Anomaly Score Threshold {thresh}:')
    print(len(ts_no_label_hourly.query(f"Outliers == 1 & Score <= {thresh}")))

In [20]:
def anomalies_plot(threshold=0.1):
    outliers_trace = go.Scatter(
    x=ts_no_label_hourly.query(f"Outliers == 1 & Score <= {threshold}")['timestamp'],
    y=ts_no_label_hourly.query(f"Outliers == 1 & Score <= {threshold}")['value'],
    mode='markers',
    marker=dict(size=10, color='red'),
    hoverinfo='text',
    text=['Date: {}<br>Weekday: {}<br>Day: {}<br>Month: {}<br>Value: {}<br>Average Value: {}<br>Outliers: {}'
          .format(date, weekday, day, month, val, avg_val, outlier) 
          for date, weekday, day, month, val, avg_val, outlier 
          in zip(ts_no_label_hourly.query("Outliers == 1")['timestamp'], 
                 ts_no_label_hourly.query("Outliers == 1")['Weekday'], 
                 ts_no_label_hourly.query("Outliers == 1")['Month_day'], 
                 ts_no_label_hourly.query("Outliers == 1")['Month'], 
                 ts_no_label_hourly.query("Outliers == 1")['value'], 
                 ts_no_label_hourly.query("Outliers == 1")['value_Average'], 
                 ts_no_label_hourly.query("Outliers == 1")['Outliers'])]
    )

    line_trace = go.Scatter(
        x=ts_no_label_hourly['timestamp'],
        y=ts_no_label_hourly['value'],
        mode='lines',
        name='Exchange Results',
        hoverinfo='text',
        text=['Date: {}<br>Weekday: {}<br>Day: {}<br>Month: {}<br>Value: {}<br>Average Value: {}'
              .format(date, weekday, day, month, val, avg_val) 
              for date, weekday, day, month, val, avg_val 
              in zip(ts_no_label_hourly['timestamp'], 
                     ts_no_label_hourly['Weekday'], 
                     ts_no_label_hourly['Month_day'], 
                     ts_no_label_hourly['Month'], 
                     ts_no_label_hourly['value'], 
                     ts_no_label_hourly['value_Average'])],

        line=dict(color='#636EFA')
    )

    data = [outliers_trace, line_trace]

    layout = go.Layout(
        title="Exchange Results Anomalies",
        height=500,
        hovermode='closest',
        showlegend=False,
        margin=dict(t=30, b=30, l=50, r=30),
    )

    fig = go.Figure(data=data, layout=layout)

    fig.show(config={'responsive': True})

In [21]:
anomalies_plot()

In [22]:
len(ts_no_label_hourly.query("Outliers == 1"))

9

In [23]:
frequencies, edges = np.histogram(score, bins=50)

histogram_trace = go.Bar(
    x=edges,
    y=frequencies,
    hoverinfo='x+y',
)

layout = go.Layout(
    title="Histogram of Scores",
    xaxis=dict(title="Score"),
    yaxis=dict(title="Frequency"),
    height=500,
    hovermode='closest',
)

fig = go.Figure(data=[histogram_trace], layout=layout)

fig.show(config={'responsive': True})

In [24]:
outliers(0.05)

Number of Outliers below Anomaly Score Threshold 0.05:
6


In [25]:
anomalies_plot(0.05)

In [26]:
outliers(0.005)

Number of Outliers below Anomaly Score Threshold 0.005:
3


In [27]:
anomalies_plot(0.005)

### 1.2 TadGAN (не зашло)

In [28]:
data = pd.read_csv('/kaggle/input/nab/realTweets/realTweets/Twitter_volume_IBM.csv', 
                   parse_dates=True)

In [29]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['timestamp'] = data['timestamp'].astype(int) // 10 ** 9

data.head()

,timestamp,value
0,1424986973,7
1,1424987273,4
2,1424987573,14
3,1424987873,6
4,1424988173,1


In [30]:
hyperparameters = {
    'mlstars.custom.timeseries_preprocessing.time_segments_aggregate#1': {
        'interval': 1000
    },
    'orion.primitives.tadgan.TadGAN#1': {
        'epochs': 5,
        'verbose': True
    }
}

orion = Orion(
    pipeline='tadgan',
    hyperparameters=hyperparameters
)

orion.fit(data)

Epoch: 1/5, Losses: {'cx_loss': -5.281, 'cz_loss': 8.2932, 'eg_loss': -3.0181}
Epoch: 2/5, Losses: {'cx_loss': -2.4031, 'cz_loss': -20.1467, 'eg_loss': 20.2696}
Epoch: 3/5, Losses: {'cx_loss': -0.6505, 'cz_loss': -4.7486, 'eg_loss': 16.8268}
Epoch: 4/5, Losses: {'cx_loss': -0.8993, 'cz_loss': 3.5893, 'eg_loss': 10.0386}
Epoch: 5/5, Losses: {'cx_loss': -0.284, 'cz_loss': 15.5998, 'eg_loss': -18.4774}


In [31]:
anomalies = orion.detect(data)
anomalies.head()

146/146 [==============================] - 2s 14ms/step


,start,end,severity
0,1426542973,1426674973,0.850717
1,1427753973,1427883973,0.749245
2,1429492973,1429632973,1.128494


In [32]:
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
anomalies['start'] = pd.to_datetime(anomalies['start'], unit='s')
anomalies['end'] = pd.to_datetime(anomalies['end'], unit='s')

fig = go.Figure()

fig.add_trace(go.Scatter(x=data['timestamp'], 
                         y=data['value'], 
                         mode='lines', 
                         name='Time Series'))

for index, row in anomalies.iterrows():
    anomaly_range = data[(data['timestamp'] >= row['start']) & (data['timestamp'] <= row['end'])]
    fig.add_trace(go.Scatter(x=anomaly_range['timestamp'], 
                             y=anomaly_range['value'], 
                             mode='lines', 
                             line=dict(color='red')))

fig.update_layout(xaxis_title='Time', 
                  yaxis_title='Value', 
                  title='Time Series with Anomalies Highlighted')
fig.show()

### 1.3 Кластеризация и KNN (если делать по минутам или секундам, то очень долго)

In [33]:
data = pd.read_csv('/kaggle/input/nab/realAdExchange/realAdExchange/exchange-4_cpc_results.csv')
data['datetime'] = pd.to_datetime(data['timestamp'])
data = data.drop(columns='timestamp')
data = data.set_index('datetime')
dd = pd.DataFrame(data['value'].resample('D').sum())

In [34]:
dd['prev'] = dd['value'].shift(periods=1)
dd['prev_7'] = dd['value'].shift(periods=7)

In [35]:
dd.dropna(inplace=True)

In [36]:
n_cls = range(1, 20)

kmeans = [KMeans(n_clusters=i, random_state=SEED).fit(dd) for i in n_cls]
scores = [kmeans[i].score(dd) for i in range(len(kmeans))]

In [37]:
n_cls_list = list(n_cls)

trace = go.Scatter(x=n_cls_list, 
                   y=scores, 
                   mode='lines')

layout = go.Layout(
    xaxis=dict(title='n_cls'),
    yaxis=dict(title='scores'),
    height=500
)

fig = go.Figure(data=[trace], layout=layout)

fig.show()

In [38]:
X = dd.values
X_std = StandardScaler().fit_transform(X)

In [39]:
mean_vec = np.mean(X_std, axis=0)
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

In [40]:
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:, i]) for i in range(len(eig_vals))]
eig_pairs.sort(key=lambda x: x[0], reverse=True)

tot = sum(eig_vals)
var_exp = [(i / tot) * 100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

In [41]:
x_values = list(range(len(var_exp)))

bar_trace = go.Bar(x=x_values, 
                   y=var_exp, 
                   name='Explained Variance')

step_trace = go.Scatter(x=x_values, 
                        y=cum_var_exp, 
                        mode='lines', 
                        name='Cumulative Explained Variance')

layout = go.Layout(
    title='Explained Variance and Cumulative Explained Variance',
    xaxis=dict(title='Principal Component'),
    yaxis=dict(title='Explained Variance'),
    height=500
)

fig = go.Figure(data=[bar_trace, step_trace], layout=layout)

fig.show()

In [42]:
def getDistanceByPoint(data, model):
    distance = pd.Series()
    for i in range(0, len(data)):
        X_a = np.array(data.loc[i])
        X_b = model.cluster_centers_[model.labels_[i] - 1]
        distance.at[i] = np.linalg.norm(X_a - X_b)
    return distance

In [43]:
data = pd.DataFrame(X_std)

In [44]:
pca = PCA(n_components=2, random_state=SEED)
data = pca.fit_transform(data)

scaler = StandardScaler()
np_scaled = scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)
data.head()

,0,1
0,-0.540879,-0.018522
1,-0.777584,0.270817
2,-0.826358,0.501881
3,-0.948150,0.547283
4,-1.076811,0.407081


In [45]:
kmeans_model = KMeans(n_clusters=7, random_state=42 * SEED).fit(data)

In [46]:
dd.reset_index(inplace=True)
dd['cluster'] = kmeans_model.predict(data)
dd.index = data.index
dd['pca1'] = data[0]
dd['pca2'] = data[1]

dist = getDistanceByPoint(data, kmeans_model)
n_outliers = int(0.05 * len(dist))
threshold = percentile(dist, 95)
dd['anomaly_cls'] = (dist >= threshold).astype(int)
dd.head()

,datetime,value,prev,prev_7,cluster,pca1,pca2,anomaly_cls
0,2011-07-08,1.710207,1.862479,1.634380,4,-0.540879,-0.018522,0
1,2011-07-09,1.387284,1.710207,1.655231,3,-0.777584,0.270817,0
2,2011-07-10,1.388968,1.387284,1.942935,3,-0.826358,0.501881,0
3,2011-07-11,1.238614,1.388968,1.843346,3,-0.948150,0.547283,0
4,2011-07-12,1.285317,1.238614,1.725985,3,-1.076811,0.407081,0


In [47]:
a = dd.loc[dd.anomaly_cls == 1, ['value', 'datetime']]

scatter_trace = go.Scatter(x=dd['datetime'], 
                           y=dd['value'], 
                           mode='lines', 
                           name='All Data')

anomaly_trace = go.Scatter(x=a['datetime'], 
                           y=a['value'], 
                           mode='markers', 
                           marker=dict(color='red'), 
                           name='Anomalies')

layout = go.Layout(
    title='Anomalies in Time Series [K-MEANS]',
    xaxis=dict(title='Datetime'),
    yaxis=dict(title='Value'),
    height=500
)

fig = go.Figure(data=[scatter_trace, anomaly_trace], layout=layout)

fig.show()

In [48]:
knn = NearestNeighbors(n_neighbors=3)
knn.fit(X)

NearestNeighbors(n_neighbors=3)

In [49]:
dist, idxs = knn.kneighbors(X)

In [50]:
dd['dist'] = dist.mean(axis=1)
threshold = percentile(dd.dist, 95)

dd['anomaly_knn'] = dd.dist > threshold
dd.head()

,datetime,value,prev,prev_7,cluster,pca1,pca2,anomaly_cls,dist,anomaly_knn
0,2011-07-08,1.710207,1.862479,1.634380,4,-0.540879,-0.018522,0,0.136552,False
1,2011-07-09,1.387284,1.710207,1.655231,3,-0.777584,0.270817,0,0.207577,False
2,2011-07-10,1.388968,1.387284,1.942935,3,-0.826358,0.501881,0,0.154348,False
3,2011-07-11,1.238614,1.388968,1.843346,3,-0.948150,0.547283,0,0.125574,False
4,2011-07-12,1.285317,1.238614,1.725985,3,-1.076811,0.407081,0,0.120064,False


In [51]:
a = dd.loc[dd['anomaly_knn'], ['value', 'datetime']]

scatter_trace = go.Scatter(x=dd['datetime'], 
                           y=dd['value'], 
                           mode='lines', 
                           name='All Data')

anomaly_trace = go.Scatter(x=a['datetime'], 
                           y=a['value'], 
                           mode='markers', 
                           marker=dict(color='red'), 
                           name='Anomalies')

layout = go.Layout(
    title='Anomalies in Time Series [KNN]',
    xaxis=dict(title='Datetime'),
    yaxis=dict(title='Value'),
    height=500
)

fig = go.Figure(data=[scatter_trace, anomaly_trace], layout=layout)

fig.show()

### 1.4 ADTK

In [53]:
s = pd.read_csv('/kaggle/input/nab/realAdExchange/realAdExchange/exchange-3_cpc_results.csv', index_col="timestamp", parse_dates=True)
s = validate_series(s)

In [54]:
min_cluster_detector = MinClusterDetector(KMeans(n_clusters=3))
anomalies_clusterization = min_cluster_detector.fit_detect(s)

In [55]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=s.index, y=s['value'], mode='lines', name='Value'))

anomaly_dates = anomalies_clusterization[anomalies_clusterization].index
fig.add_trace(go.Scatter(x=anomaly_dates, y=s.loc[anomaly_dates]['value'],
                         mode='markers', marker=dict(color='red'), name='Anomalies'))

fig.update_layout(title='Anomalies in Time Series [ADTK K-MEANS]',
                  xaxis_title='Datetime',
                  yaxis_title='Value')
fig.show()

In [56]:
outlier_detector = OutlierDetector(LocalOutlierFactor(contamination=0.01))
anomalies_outlier = outlier_detector.fit_detect(s)

In [57]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=s.index, y=s['value'], mode='lines', name='Value'))

anomaly_dates = anomalies_outlier[anomalies_outlier].index
fig.add_trace(go.Scatter(x=anomaly_dates, y=s.loc[anomaly_dates]['value'],
                         mode='markers', marker=dict(color='red'), name='Anomalies'))

fig.update_layout(title='Anomalies in Time Series [ADTK OutlierDetector]',
                  xaxis_title='Datetime',
                  yaxis_title='Value')
fig.show()

In [58]:
pca_ad = PcaAD(k=1)
anomalies_pca = pca_ad.fit_detect(s)

In [59]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=s.index, y=s['value'], mode='lines', name='Value'))

anomaly_dates = anomalies_pca[anomalies_pca].index
fig.add_trace(go.Scatter(x=anomaly_dates, y=s.loc[anomaly_dates]['value'],
                         mode='markers', marker=dict(color='red'), name='Anomalies'))

fig.update_layout(title='Anomalies in Time Series [ADTK PCA]',
                  xaxis_title='Datetime',
                  yaxis_title='Value')
fig.show()

## Supervised подходы

### 2.1 Autoencoder Keras

In [60]:
# timeseries без аномалий для обучения
df_small_noise = pd.read_csv('/kaggle/input/nab/artificialNoAnomaly/artificialNoAnomaly/art_daily_small_noise.csv', 
                             parse_dates=True, 
                             index_col="timestamp"
)

# timeseries с аномалиями для теста
df_daily_jumpsup = pd.read_csv('/kaggle/input/nab/artificialWithAnomaly/artificialWithAnomaly/art_daily_jumpsup.csv', 
                               parse_dates=True, 
                               index_col="timestamp"
)

In [61]:
fig = go.Figure()

for column in df_small_noise.columns:
    fig.add_trace(go.Scatter(x=df_small_noise.index, y=df_small_noise[column], mode='lines', name=column))

fig.update_layout(title='art_daily_small_noise',
                  xaxis_title='Timestamp',
                  yaxis_title='Value')

fig.show()

In [62]:
fig = go.Figure()

for column in df_daily_jumpsup.columns:
    fig.add_trace(go.Scatter(x=df_daily_jumpsup.index, y=df_daily_jumpsup[column], mode='lines', name=column))

fig.update_layout(title='art_daily_jumpsup',
                  xaxis_title='Timestamp',
                  yaxis_title='Value')

fig.show()

In [63]:
training_mean = df_small_noise.mean()
training_std = df_small_noise.std()
df_training_value = (df_small_noise - training_mean) / training_std

print("Number of training samples:", len(df_training_value))

Number of training samples: 4032


In [64]:
TIME_STEPS = 288


def create_sequences(values, time_steps=TIME_STEPS):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)


x_train = create_sequences(df_training_value.values)
print("Training input shape: ", x_train.shape)

Training input shape:  (3745, 288, 1)


In [65]:
model = keras.Sequential(
    [
        layers.Input(shape=(x_train.shape[1], x_train.shape[2])),
        layers.Conv1D(
            filters=32,
            kernel_size=7,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Dropout(rate=0.2),
        layers.Conv1D(
            filters=16,
            kernel_size=7,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Conv1DTranspose(
            filters=16,
            kernel_size=7,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Dropout(rate=0.2),
        layers.Conv1DTranspose(
            filters=32,
            kernel_size=7,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Conv1DTranspose(filters=1, kernel_size=7, padding="same"),
    ]
)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 144, 32)           256       
                                                                 
 dropout_6 (Dropout)         (None, 144, 32)           0         
                                                                 
 conv1d_5 (Conv1D)           (None, 72, 16)            3600      
                                                                 
 conv1d_transpose (Conv1DTr  (None, 144, 16)           1808      
 anspose)                                                        
                                                                 
 dropout_7 (Dropout)         (None, 144, 16)           0         
                                                                 
 conv1d_transpose_1 (Conv1D  (None, 288, 32)           3616      
 Transpose)                                             

In [66]:
history = model.fit(
    x_train,
    x_train,
    epochs=50,
    batch_size=128,
    validation_split=0.1,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")
    ],
)

Epoch 1/50
27/27 [==============================] - 3s 76ms/step - loss: 0.5515 - val_loss: 0.3403
Epoch 2/50
27/27 [==============================] - 2s 64ms/step - loss: 0.1499 - val_loss: 0.0491
Epoch 3/50
27/27 [==============================] - 2s 64ms/step - loss: 0.0596 - val_loss: 0.0357
Epoch 4/50
27/27 [==============================] - 2s 65ms/step - loss: 0.0486 - val_loss: 0.0307
Epoch 5/50
27/27 [==============================] - 2s 67ms/step - loss: 0.0423 - val_loss: 0.0295
Epoch 6/50
27/27 [==============================] - 2s 64ms/step - loss: 0.0374 - val_loss: 0.0281
Epoch 7/50
27/27 [==============================] - 2s 65ms/step - loss: 0.0335 - val_loss: 0.0268
Epoch 8/50
27/27 [==============================] - 2s 70ms/step - loss: 0.0303 - val_loss: 0.0265
Epoch 9/50
27/27 [==============================] - 2s 69ms/step - loss: 0.0278 - val_loss: 0.0268
Epoch 10/50
27/27 [==============================] - 2s 65ms/step - loss: 0.0260 - val_loss: 0.0271
Epoch 11/

In [67]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=history.history["loss"], mode='lines', name='Training Loss'))
fig.add_trace(go.Scatter(y=history.history["val_loss"], mode='lines', name='Validation Loss'))

fig.update_layout(
    title="Training and Validation Loss",
    xaxis_title="Epoch",
    yaxis_title="Loss"
)

fig.show()

In [68]:
x_train_pred = model.predict(x_train)
train_mae_loss = np.mean(np.abs(x_train_pred - x_train), axis=1)

histogram = go.Histogram(x=[loss for [loss] in train_mae_loss])

layout = go.Layout(title='Histogram of Train MAE Loss', xaxis=dict(title='Train MAE loss'), yaxis=dict(title='Num of samples'))

fig = go.Figure(data=[histogram], layout=layout)

fig.show()

threshold = np.max(train_mae_loss)
print("Reconstruction error threshold: ", threshold)

118/118 [==============================] - 1s 8ms/step


Reconstruction error threshold:  0.06985214704560011


In [69]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(len([x for [x] in x_train[0]]))), 
                         y=[x for [x] in x_train[0]],
                         mode='lines',
                         name='x_train'))

fig.add_trace(go.Scatter(x=list(range(len([x for [x] in x_train_pred[0]]))), 
                         y=[x for [x] in x_train_pred[0]],
                         mode='lines',
                         name='x_train_pred'))

fig.show()

In [70]:
test_mean = df_daily_jumpsup.mean()
test_std = df_daily_jumpsup.std()

df_test_value = (df_daily_jumpsup - training_mean) / training_std

In [71]:
x_test = create_sequences(df_test_value.values)
print("Test input shape: ", x_test.shape)

x_test_pred = model.predict(x_test)
test_mae_loss = np.mean(np.abs(x_test_pred - x_test), axis=1)
test_mae_loss = test_mae_loss.reshape((-1))

fig = go.Figure(data=[go.Histogram(x=test_mae_loss, nbinsx=50)])

fig.update_layout(title='Histogram Test MAE Loss',
                  xaxis_title='Test MAE loss',
                  yaxis_title='No of samples')

fig.show()

Test input shape:  (3745, 288, 1)
118/118 [==============================] - 1s 7ms/step


In [72]:
anomalies = test_mae_loss > threshold
print("Number of anomaly samples: ", np.sum(anomalies))
print("Indices of anomaly samples: ", np.where(anomalies))

Number of anomaly samples:  407
Indices of anomaly samples:  (array([ 217,  778,  790,  793,  794,  795, 1945, 1946, 2521, 2698, 2699,
       2701, 2702, 2703, 2704, 2705, 2706, 2707, 2708, 2709, 2710, 2711,
       2712, 2713, 2714, 2715, 2716, 2717, 2718, 2719, 2720, 2721, 2722,
       2723, 2724, 2725, 2726, 2727, 2728, 2729, 2730, 2731, 2732, 2733,
       2734, 2735, 2736, 2737, 2738, 2739, 2740, 2741, 2742, 2743, 2744,
       2745, 2746, 2747, 2748, 2749, 2750, 2751, 2752, 2753, 2754, 2755,
       2756, 2757, 2758, 2759, 2760, 2761, 2762, 2763, 2764, 2765, 2766,
       2767, 2768, 2769, 2770, 2771, 2772, 2773, 2774, 2775, 2776, 2777,
       2778, 2779, 2780, 2781, 2782, 2783, 2784, 2785, 2786, 2787, 2788,
       2789, 2790, 2791, 2792, 2793, 2794, 2795, 2796, 2797, 2798, 2799,
       2800, 2801, 2802, 2803, 2804, 2805, 2806, 2807, 2808, 2809, 2810,
       2811, 2812, 2813, 2814, 2815, 2816, 2817, 2818, 2819, 2820, 2821,
       2822, 2823, 2824, 2825, 2826, 2827, 2828, 2829, 2830, 2

In [73]:
anomalous_data_indices = []
for data_idx in range(TIME_STEPS - 1, len(df_test_value) - TIME_STEPS + 1):
    if np.all(anomalies[data_idx - TIME_STEPS + 1 : data_idx]):
        anomalous_data_indices.append(data_idx)

In [74]:
df_subset = df_daily_jumpsup.iloc[anomalous_data_indices]

In [75]:
fig = go.Figure()

for column in df_daily_jumpsup.columns:
    fig.add_trace(go.Scatter(x=df_daily_jumpsup.index, y=df_daily_jumpsup[column], mode='lines', name=column, line=dict(color='blue')))

for column in df_subset.columns:
    fig.add_trace(go.Scatter(x=df_subset.index, y=df_subset[column], mode='lines', name=column, line=dict(color='red')))

fig.update_layout(title='Anomalies',
                  xaxis_title='Timestamp',
                  yaxis_title='Value')

fig.show()